<a href="https://colab.research.google.com/github/amanglarus/image_classification_using-tensorflow/blob/main/classification_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount(‘/content/drive’)

SyntaxError: ignored

In [ ]:
!cp 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [ ]:
NUM_CLASSES = 2
BATCH_SIZE=32
IMAGE_SIZE = 48
NUM_CHANNEL = 3

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir='/home/madan/Documents/dataset/Train'
val_data_dir='/home/madan/Documents/dataset/Val'

from keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator(rescale=1./255)


IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
MODEL_IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNEL)

train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical')

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode='categorical')






In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import MobileNetV2

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=MODEL_IMAGE_SHAPE,
    alpha=1.0,
    include_top=False,
    #weights='imagenet',
    #input_tensor=None,
    #pooling=None,
    #classes=1000,
    #classifier_activation='softmax',
    #**kwargs
)

for layer in base_model.layers: #base_model.layers[:-2]
    layer.trainable = True  



# penultimate_layer = model.layers[-2]
# new_top_layer = tf.keras.layers.Dense(1)(penultimate_layer.output)  # create new FC layer and connect it to the rest of the model
# new_model = tf.keras.models.Model(model.input, new_top_layer)



# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.layers.Dense(64, activation="relu"),
    tf.keras.layers.layers.Dropout(0.2),
    tf.keras.layers.layers.Dense(NUM_CLASSES, activation="softmax"),
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("image_classification_checkpoint.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[checkpoint,earlystop]

main_model.compile(loss='SparseCategoricalCrossentropy',
                   optimizer=Adam(learning_rate=0.0015),
                   metrics=['accuracy'])

epochs=50

batch_size = 32

history = main_model.fit(train_generator,
                        #  steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                        #  callbacks=callbacks,
                         validation_data=val_generator,
                        #  validation_steps=num_val_samples//batch_size
                         )